# 元々のjsonファイルを取得し，整形したものを出力

・ラベル名をtagsに統一

・ラベルとして使用されていた数値IDを文字型に変換

In [ ]:
#{"words": ["CHICAGO", "AT", "ATLANTA"], "ner": ["B-ORG", "O", "B-LOC"]}

In [5]:
import json

In [19]:
with open("data/original/id2coarse_tags.json","r") as f:
    label_coarse =json.loads(f.read())
    
with open("data/original/id2fine_tags.json","r") as f:
    label_fine =json.loads(f.read())

In [22]:
def coarse_str_tag(tags):
    new_tag = [label_coarse[str(i)] for i in tags]
    return new_tag
    
def fine_str_tag(tags):
    new_tag = [label_fine[str(i)] for i in tags]
    return new_tag

In [23]:
for filename in ["train","test","dev"]:
    
    data = []
    with open("data/original/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))

    coarse = []
    for d in data:
        new_d = {}
        new_d["tokens"] = d["tokens"]
        new_d["tags"] = coarse_str_tag(d["coarse_tags"])
        coarse.append(new_d)
    with open("data/coarse/"+filename+".json","w") as f:
        for d in coarse:
            f.write(json.dumps(d))
            f.write("\n")

    fine = []
    for d in data:
        new_d = {}
        new_d["tokens"] = d["tokens"]
        new_d["tags"] = fine_str_tag(d["fine_tags"])
        fine.append(new_d)
    with open("data/fine/"+filename+".json","w") as f:
        for d in fine:
            f.write(json.dumps(d))
            f.write("\n")

In [7]:
label_coarse

[{'0': 'O',
  '1': 'art',
  '2': 'building',
  '3': 'event',
  '4': 'location',
  '5': 'organization',
  '6': 'other',
  '7': 'person',
  '8': 'product'}]

# Simple-BI

・BIラベルのみに着目したデータ拡張

・交換率100％

In [1]:
import json
import random

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def Simple_BI(dataname,filename,n):
    data = []
    with open("data/"+dataname+"/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))
            
    alterna_labels = {}
    if dataname == "c":
        tag_json = "data/original/id2coarse_tags.json"
    else:
        tag_json = "data/original/id2fine_tags.json"
    with open(tag_json,"r") as f:
        label_coarse =json.loads(f.read())
        label_list = list(label_coarse.values())[1:]
    for label in label_list:
        alterna_labels[label] = set()
    for d in data:
        for token,tag in zip(d["tokens"],d["tags"]):
            if tag in label_list:
                alterna_labels[tag].add(token)

    data_f_path = "data/"+dataname+"_BI_x"+str(n)+"/"+filename+".json"
    memo_f_path = "log/"+dataname+"_BI_x"+str(n)+"_"+filename+"_memo.txt"
    
    
    with open(data_f_path,"w") as data_f, open(memo_f_path,"w") as memo_f:
        for d in data:
            data_f.write(json.dumps(d))
            data_f.write("\n")

        add_n = n*len(data)-len(data)
        print()
        print(dataname,filename,"add_n",add_n)
        
        for now in range(add_n):
            if now%10 == 0:
                print('\r%d / %d' %(now, add_n), end='')

            x = random.randint(0,len(data)-1)
            d = data[x]
            new_d = dict()
            new_d["tokens"] = []
            new_d["tags"] = data[x]["tags"]

            cnt = 0

            for token,tag in zip(d["tokens"],d["tags"]):
                if tag != "O":
                    if tag in alterna_labels:
                        token=random_choice_token(alterna_labels[tag]-{token})
                        cnt = cnt + 1
                    new_d["tokens"].append(token)

            data_f.write(json.dumps(new_d))
            data_f.write("\n")

            memo_f.write(str(x)+"\t"+" ".join(d["tokens"])+"\n")
            memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d["tokens"])+"\n")
            memo_f.write("\n")

        memo_f.write(str(cnt)+"sentences augmented!!!! \n")

In [6]:
# for dataname in ["f"]:
#     for filename in ["dev","train"]:
#         Simple_BI(dataname,filename,2)
#         Simple_BI(dataname,filename,5)
#         Simple_BI(dataname,filename,10)

# TreePos-BI

・ 形態素情報付与

In [1]:
from nltk.parse.stanford import *
import nltk
from nltk.tree import *
import pickle
import os

In [2]:
java_path = "jre1.8.0_333/bin/java.exe"
os.environ['JAVAHOME'] = java_path

parser =  'stanford-corenlp/stanford-parser-full-2020-11-17/stanford-parser.jar'
parser_model = 'stanford-corenlp/stanford-corenlp-4.2.0-models-english.jar'

#POSタグの分析用
pos = StanfordParser(path_to_jar=parser, path_to_models_jar = parser_model)
#係り受け関係の分析用
dep_parser = StanfordDependencyParser(path_to_jar=parser, path_to_models_jar = parser_model)

def POSTagAnalysis(text):
    out = pos.raw_parse(text)
    out = list(out)
    tree = out[0]
    return tree

/tmp/ipykernel_269207/2297462148.py:8: DeprecationWarning: The StanfordParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPParser instead.
  pos = StanfordParser(path_to_jar=parser, path_to_models_jar = parser_model)
/tmp/ipykernel_269207/2297462148.py:10: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  dep_parser = StanfordDependencyParser(path_to_jar=parser, path_to_models_jar = parser_model)


In [46]:
import json
p = 127001#止まったところからスタート

def TreePOS(filename):
    data = []
    with open("data/f/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))
            
    new_document=[]
    document_trees = []
    c = p
    
    #タプル型で複数引数を渡す
    for s in data[p:]:
        
        text = " ".join(s["tokens"])
        tree = POSTagAnalysis(text)
        document_trees.append(tree)
        new_d = dict()
        new_d["tokens"] = []
        new_d["tree_pos"] = []
        new_d["tags"] = []

        for i in range(len(tree.pos())):
            new_d["tokens"].append(tree.pos()[i][0])
            new_d["tree_pos"].append(tree.pos()[i][1])

        new_d["tags"]=s["tags"]
        new_document.append(new_d)
        del tree
        
        print('\r%d / %d' %(c, len(data)), end='')
        
        if (c % 1000 == 0 and c!=0) or c == len(data):
            with open("data/Stanford_coreNLP/"+filename+"_"+str(c)+"_tree.binaryfile", 'wb') as f:
                pickle.dump(document_trees,f)
            with open("data/Stanford_coreNLP/"+filename+"_"+str(c)+"_data.binaryfile", 'wb') as f:
                pickle.dump(new_document,f)
            
            print("\nSAVED ==> "+filename+"_"+str(c)+"_tree.binaryfile")
            print("SAVED ==> "+filename+"_"+str(c)+"_data.binaryfile")
        c = c+1

In [47]:
for filename in ["train"]:
# for filename in ["dev"]:
    TreePOS(filename)

128000 / 131766
SAVED ==> train_128000_tree.binaryfile
SAVED ==> train_128000_data.binaryfile
129000 / 131766
SAVED ==> train_129000_tree.binaryfile
SAVED ==> train_129000_data.binaryfile
130000 / 131766
SAVED ==> train_130000_tree.binaryfile
SAVED ==> train_130000_data.binaryfile
131000 / 131766
SAVED ==> train_131000_tree.binaryfile
SAVED ==> train_131000_data.binaryfile
131765 / 131766

・pickleファイルを統合

・以下のtokensを持つデータに対してエラーが出現したため(tokensとtagsの長さ違い)，errorを避けて保存

['The',  'Philippine',  'peso',  'sign',  '-LRB-',  '-RRB-',  'is',  'the',  'currency',  'symbol',  'used',  'for',  'Philippine',  'peso',  ',',  'the',  'official',  'currency',  'of',  'the',  'Philippines',  '.']

In [72]:
import os
for path_sub in ["dev","train_1-51k","train_52-81k","train_82-131k"]
    path = "data/Stanford_coreNLP/"
    files = os.listdir(path+path_sub+"/")
    data_list = []
    tree_list = []

    for filename in files:
        with open(path+path_sub+"/"+filename,'rb') as f:
            data = pickle.load(f)
        if "data" in filename:
            data_list = data_list + data
        else:
            tree_list = tree_list + data

    with open(path+path_sub+"_data.binaryfile", 'wb') as f:
        pickle.dump(data_list,f)

    with open(path+path_sub+"_tree.binaryfile", 'wb') as f:
        pickle.dump(tree_list,f)

In [112]:
error_data = []
for path_sub in ["dev"]:
    path = "data/Stanford_coreNLP/"

    with open(path+path_sub+"_data.binaryfile",'rb') as rf:
        data = pickle.load(rf)
        
    new_data = []
    
    for d in data:
        new_d = dict()
        new_d["tokens"] = d["tokens"]
        try:
            new_d["tags"] = [str(d["tree_pos"][j])+"_"+str(d["tags"][j]) for j in range(len(d["tags"]))]
            new_data.append(new_d)
            
        except:
            error_data.append(d)
            
    with open(path + "dev.json","w") as wf:
        for d in new_data:
            wf.write(json.dumps(d))
            wf.write("\n")
            
    with open(path + "error_dev.json", 'w') as ef:
        for d in error_data:
            ef.write(json.dumps(d))
            ef.write("\n")

In [113]:
error_data = []
data = []
path = "data/Stanford_coreNLP/"

for path_sub in ["train_1-51k","train_52-81k","train_82-131k"]:
    with open(path+path_sub+"_data.binaryfile",'rb') as rf:
        addition = pickle.load(rf)
    data= data+addition
        
new_data = []
for d in data:
    new_d = dict()
    new_d["tokens"] = d["tokens"]
    try:
        new_d["tags"] = [str(d["tree_pos"][j])+"_"+str(d["tags"][j]) for j in range(len(d["tags"]))]
        new_data.append(new_d)

    except:
        error_data.append(d)

with open(path + "train.json","w") as wf:
    for d in new_data:
        wf.write(json.dumps(d))
        wf.write("\n")

with open(path + "error_train.json", 'w') as ef:
    for d in error_data:
        ef.write(json.dumps(d))
        ef.write("\n")

・データ拡張実装

In [115]:
train_data = []
with open("data/Stanford_coreNLP/train.json","r") as f:
    for line in f:
        train_data.append(json.loads(line))

dev_data = []
with open("data/Stanford_coreNLP/dev.json","r") as f:
    for line in f:
        dev_data.append(json.loads(line))

In [ ]:

alterna_labels = {}
if dataname == "c":
    tag_json = "data/original/id2coarse_tags.json"
else:
    tag_json = "data/original/id2fine_tags.json"
with open(tag_json,"r") as f:
    label_coarse =json.loads(f.read())
    label_list = list(label_coarse.values())[1:]
for label in label_list:
    alterna_labels[label] = set()
for d in data:
    for token,tag in zip(d["tokens"],d["tags"]):
        if tag in label_list:
            alterna_labels[tag].add(token)

In [ ]:
import json
import random

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def TreePos_BI(dataname,filename,n):
    data = []
    with open("data/"+dataname+"/"+filename+".json","r") as f:
        for line in f:
            data.append(json.loads(line))
            
    alterna_labels = {}
    #~~~~~~~#
    data_f_path = "data/"+dataname+"_BI_x"+str(n)+"/"+filename+".json"
    memo_f_path = "log/"+dataname+"_BI_x"+str(n)+"_"+filename+"_memo.txt"
    
    
    with open(data_f_path,"w") as data_f, open(memo_f_path,"w") as memo_f:
        for d in data:
            data_f.write(json.dumps(d))
            data_f.write("\n")

        add_n = n*len(data)-len(data)
        print()
        print(dataname,filename,"add_n",add_n)
        
        for now in range(add_n):
            if now%10 == 0:
                print('\r%d / %d' %(now, add_n), end='')

            x = random.randint(0,len(data)-1)
            d = data[x]
            new_d = dict()
            new_d["tokens"] = []
            new_d["tags"] = data[x]["tags"]

            cnt = 0

            for token,tag in zip(d["tokens"],d["tags"]):
                if tag != "O":
                    if tag in alterna_labels:
                        token=random_choice_token(alterna_labels[tag]-{token})
                        cnt = cnt + 1
                    new_d["tokens"].append(token)

            data_f.write(json.dumps(new_d))
            data_f.write("\n")

            memo_f.write(str(x)+"\t"+" ".join(d["tokens"])+"\n")
            memo_f.write(" "*len(str(x))+"\t"+" ".join(new_d["tokens"])+"\n")
            memo_f.write("\n")

        memo_f.write(str(cnt)+"sentences augmented!!!! \n")